# Guild AI Quick Start

This Notebook mirrors the steps in [Guild AI Quick Start](https://guild.ai/docs/start/):

- Part 1 - Experiment basics
    - Create a mock training function
    - Generate a run with Guild
    - Examine the run
    - Generate a second run
    - Compare runs
- Part 2 - Hyperparameter optimization

## Notebooks config

Modify the variables below to change Notebook configuration.

`GULID_HOME` is the location of generated runs. By default it is the location subdirectory "guild-env". Note that we initialize this directory below by permanently deleting any runs it contains before proceeding. If you don't want to delete runs in `GUILD_HOME`, set `DELETE_RUNS_ON_INIT` to `False` below.

In [150]:
GUILD_HOME = "guild-env"

`DELETE_RUNS_ON_INIT` determines whether or not runs are initially deleted from `GUILD_HOME` below. As this Notebook is for demonstration purposes, it's usually a good idea to delete any existing runs before proceeding. To prevent any runs from being deleted, set `DELETE_RUNS_ON_INIT` to `False`.

In [151]:
DELETE_RUNS_ON_INIT = True

## Guild interactive Python API

Guild AI functionality is available through a Notebook compatible interface defined in `guild.ipy`.

Import this module as `guild`:

In [188]:
import guild.ipy as guild

## Initialize Guild Home

Guild home is a directory containing the runs that Guild generates. This Notebook uses the directory defined by `GUILD_HOME`. Ensure that the directory exists and use `set_guild_home()` to set the value. 

In [153]:
import os

if not os.path.exists(GUILD_HOME):
    os.mkdir("guild-env")
    
guild.set_guild_home("guild-env")

Clear the director of any runs.

In [154]:
if DELETE_RUNS_ON_INIT:
    deleted = guild.runs().delete(permanent=True)
    print("Deleted %i run(s)" % len(deleted))

Deleted 3 run(s)


## Mock training function

Create a mock training script. This function doesn’t actually train anything, but simulates the training process of accepting hyperparameters as inputs and generating a loss.

In [174]:
import numpy as np

def train(x=0.1, noise=0.1):
    loss = (np.sin(5 * x) * (1 - np.tanh(x ** 2)) + np.random.randn() * noise)
    print("loss: %f" % loss)

Run the mock training function a couple times to see how it works.

**NOTE:** The function uses a random component to simulate training "noise". This causes the results to differ across runs.

In [175]:
print("Trial 1:")
train(x=-2.0)

Trial 1:
loss: -0.055170


In [176]:
print("Trial 2:")
train(x=0.0)

Trial 2:
loss: -0.050035


In [177]:
print("Trial 3:")
train(x=2.0)

Trial 3:
loss: 0.015354


## Generate a run

Run the mock train function using Guild.

In [178]:
run, return_val = guild.run(train)

loss: 0.654873


The `run()` function returns a tuple of the Guild run and the return value of the function.

In this case, our mock training function doesn't return a value.

In [179]:
print(return_val)

None


The `run` variable is a Guild object that represents the run.

## Examine the run

We can work with the run object directly to get information about it.

Each run has a uniqiue ID:

In [180]:
run.id

'6fcca5d6778511e9950fc85b764bbf34'

Each run is associated with a unique directory, which includes the ID. The run directory contains metadata and output associated with the run.

In [181]:
run.dir

'guild-env/runs/6fcca5d6778511e9950fc85b764bbf34'

Runs have *attributes*. You can list the attribute names using `attr_names()`:

In [182]:
run.attr_names()

['exit_status', 'flags', 'id', 'initialized', 'started', 'stopped']

Read an attribute value using `get()`. For example, to read the *flags* attribute, use:

In [183]:
run.get("flags")

{'noise': 0.1, 'x': 0.1}

Flags are values provided to the function. In this case, both values are provided as function keyword default values. Later you run `train` using different flag values.

Each run has a *status*, which indicates if the run is still running and whether or not it completed successfully or terminated with an error.

In [184]:
run.status

'completed'

## List runs

List runs using `guild.runs()`:

In [185]:
runs = guild.runs()
runs

,run,operation,started,status,label
0,6fcca5d6,train(),2019-05-15 21:51:00,completed,
1,f836e0cc,train(),2019-05-15 21:47:39,completed,


Each time you run an operation, the run appears in this list.

## Print run info

From `runs`, we can print information for the latest run using `runs.info()`.

In [186]:
runs.info()

id: 6fcca5d6778511e9950fc85b764bbf34
operation: train()
status: completed
started: 2019-05-15 21:51:00
stopped: 2019-05-15 21:51:00
label: 
run_dir: ~/SCM/guild-examples/notebooks/guild-env/runs/6fcca5d6778511e9950fc85b764bbf34
flags:
  noise: 0.1
  x: 0.1


This information reflects the information you saw in the prior section.

## Run scalars

Run results are recorded as *scalars*. You can list scalars in two ways.

Specify the `scalars` flag to `info()`:

In [187]:
runs.info(scalars=True)

id: 6fcca5d6778511e9950fc85b764bbf34
operation: train()
status: completed
started: 2019-05-15 21:51:00
stopped: 2019-05-15 21:51:00
label: 
run_dir: ~/SCM/guild-examples/notebooks/guild-env/runs/6fcca5d6778511e9950fc85b764bbf34
flags:
  noise: 0.1
  x: 0.1
scalars:
  loss: 0.654873 (step 0)


The other way returns a data frame for the scalars:

In [169]:
scalars = runs.scalars()
scalars

,avg_val,count,first_step,first_val,last_step,last_val,max_step,max_val,min_step,min_val,prefix,run,tag,total
0,0.274548,1,0,0.274548,0,0.274548,0,0.274548,0,0.274548,,f836e0cc778411e9950fc85b764bbf34,loss,0.274548
1,0.100000,1,0,0.100000,0,0.100000,0,0.100000,0,0.100000,,f836e0cc778411e9950fc85b764bbf34,noise,0.100000
2,0.100000,1,0,0.100000,0,0.100000,0,0.100000,0,0.100000,,f836e0cc778411e9950fc85b764bbf34,x,0.100000


Guild stores aggregates of each tag, including first, last, max, min, and average.

Here's the last recorded `loss` for the last run.

In [126]:
scalars.query("(run == '%s') and (tag == 'loss')" % run.id)["last_val"]

0    0.4652
Name: last_val, dtype: float64

## Generate a second run

Run train again with different flags.

In [147]:
run, _ = guild.run(train, x=0.2, _label="whoop")

x: 0.200000
noise: 0.100000
loss: 0.922072


List runs:

In [148]:
guild.runs()

,run,operation,started,status,label
0,7f68fe1a,train(),2019-05-15 21:37:07,completed,whoop
1,7d129810,train(),2019-05-15 21:01:15,completed,hello
2,c4768ee2,train(),2019-05-15 20:56:06,completed,


## Compare runs

In [149]:
guild.runs().compare()

,run,operation,time,status,label,noise,x,step,loss
0,7f68fe1a,train(),0:00:00,completed,whoop,0.1,0.2,0,0.922072
1,7d129810,train(),0:00:00,completed,hello,0.1,0.2,0,1.017215
2,c4768ee2,train(),0:00:00,completed,,0.1,0.1,0,0.465200
